## Exercise 10.1 Numerical Solution for Project Sub-Contracting

Follow the example in the handout to incrementally produce a version of the Gurobi code that does not hard-code in numbers but obtain them from appropriate data structures.

**Decision variable:**

- Let $x_i$ denote whether to schedule job $i$ for own company. (Binary)
- Let $y_i$ denote whether to subcontract job $i$. (Binary)

**Objective:**

$$\text{Maximize:} \qquad 30x_1+10x_2+26x_3+18x_4+20x_5+6y_1 +2y_2+8y_3+9y_4+4y_5 $$


**Constraints:**

$$\begin{aligned}
\text{(Labor)} && 1300x_1+950x_2+1000x_3+1400x_4+1600x_5 & \le 4800 \\
\text{(Doing every project)} && x_1 + y_1 & = 1 \\
&& x_2 + y_2 & = 1 \\
&& x_3 + y_3 & = 1 \\
&& x_4 + y_4 & = 1 \\
&& x_5 + y_5 & = 1 \\
\end{aligned}$$


### Version 1: Hard-coding in everything

For comparison purposes, write a version of the code that hard-codes in everything, similar to version 1 of the previous example. Remember to set the sense in the objective to GRB.MAXIMIZE.

In [1]:
# Write your code here
from gurobipy import Model, GRB
mod=Model()
x1=mod.addVar(vtype=GRB.BINARY)
x2=mod.addVar(vtype=GRB.BINARY)
x3=mod.addVar(vtype=GRB.BINARY)
x4=mod.addVar(vtype=GRB.BINARY)
x5=mod.addVar(vtype=GRB.BINARY)
y1=mod.addVar(vtype=GRB.BINARY)
y2=mod.addVar(vtype=GRB.BINARY)
y3=mod.addVar(vtype=GRB.BINARY)
y4=mod.addVar(vtype=GRB.BINARY)
y5=mod.addVar(vtype=GRB.BINARY)
mod.setObjective(30*x1+10*x2+26*x3+18*x4+20*x5+6*y1+2*y2+8*y3+9*y4+4*y5,sense = GRB.MAXIMIZE)
mod.addConstr(1300*x1+950*x2+1000*x3+1400*x4+1600*x5<=4800)
mod.addConstr(x1+y1==1)
mod.addConstr(x2+y2==1)
mod.addConstr(x3+y3==1)
mod.addConstr(x4+y4==1)
mod.addConstr(x5+y5==1)
mod.setParam('OutputFlag',False)
mod.optimize()
print('Optimal objective:',mod.objVal)
print(f'Optimal solution: x1={x1.x}, x2={x2.x}, x3={x3.x}, x4={x4.x}, x5={x5.x}')

Academic license - for non-commercial use only - expires 2022-10-02
Using license file C:\Users\jainf\gurobi.lic
Optimal objective: 88.0
Optimal solution: x1=1.0, x2=1.0, x3=1.0, x4=1.0, x5=0.0


In [2]:
# Sample output

### Version 2: Using addVars to create multiple variables at once

Using addVars, generate all of the x's using one command, and all of the y's using one command. Also, make the optimal solution easier to read, as in the output below.

In [3]:
# Write your code here
# Write your code here
from gurobipy import Model, GRB
mod=Model()
jobs = range(1,6)
x=mod.addVars(jobs, vtype=GRB.BINARY)
y=mod.addVars(jobs, vtype=GRB.BINARY)
mod.setObjective(30*x[1]+10*x[2]+26*x[3]+18*x[4]+20*x[5]+6*y[1]+2*y[2]+8*y[3]+9*y[4]+4*y[5],sense = GRB.MAXIMIZE)
mod.addConstr(1300*x[1]+950*x[2]+1000*x[3]+1400*x[4]+1600*x[5]<=4800)
mod.addConstr(x[1]+y[1]==1)
mod.addConstr(x[2]+y[2]==1)
mod.addConstr(x[3]+y[3]==1)
mod.addConstr(x[4]+y[4]==1)
mod.addConstr(x[5]+y[5]==1)
mod.setParam('OutputFlag',False)
mod.optimize()
print('Optimal objective:',mod.objVal)
print('Optimal solution: do projects ',end='')
for j in jobs:
    if x[j].x==1:
        print(j, end=' ')
print('yourself')

Optimal objective: 88.0
Optimal solution: do projects 1 2 3 4 yourself


In [2]:
# Sample output

Optimal objective: 88.0
Optimal solution: do projects 1 2 3 4 yourself


### Version 3 and 4:  Using list comprehension and for loops

Instead of hard-coding in the numbers, obtain them from the following data structures. Moreover, use list comprehension to generate the large sums, and for loops to generate the repetitive constraints. Build up the formulation part by part and double in the end check by making Gurobi display the entire concrete formulation.

In [4]:
import pandas as pd
projects=range(1,6)
ownLabor=4800
profit=pd.DataFrame([[30,10,26,18,20],[6,2,8,9,4]], \
                    index=['Yourself','Subcontract'], columns=projects)
profit

,1,2,3,4,5
Yourself,30,10,26,18,20
Subcontract,6,2,8,9,4


In [5]:
laborRequired=pd.Series([1300,950,1000,1400,1600],index=projects)
laborRequired

1    1300
2     950
3    1000
4    1400
5    1600
dtype: int64

In [6]:
# Objective function (Write your code here)
from gurobipy import Model, GRB
mod=Model()
jobs = range(1,6)
x=mod.addVars(jobs, vtype=GRB.BINARY, name='x')
y=mod.addVars(jobs, vtype=GRB.BINARY, name='y')
mod.update()
sum(i*x[j] for i,j in zip(profit.iloc[0],jobs)) + sum(i*y[j] for i,j in zip(profit.iloc[1],jobs))



<gurobi.LinExpr: 30.0 x[1] + 10.0 x[2] + 26.0 x[3] + 18.0 x[4] + 20.0 x[5] + 6.0 y[1] + 2.0 y[2] + 8.0 y[3] + 9.0 y[4] + 4.0 y[5]>

In [5]:
# Sample output

<gurobi.LinExpr: 30.0 x[1] + 6.0 y[1] + 10.0 x[2] + 2.0 y[2] + 26.0 x[3] + 8.0 y[3] + 18.0 x[4] + 9.0 y[4] + 20.0 x[5] + 4.0 y[5]>

In [7]:
# Labor constraint (Write your code here)
sum(b*x[j] for b,j in zip(laborRequired,jobs))<=4800

<gurobi.TempConstr: <gurobi.LinExpr: 1300.0 x[1] + 950.0 x[2] + 1000.0 x[3] + 1400.0 x[4] + 1600.0 x[5]> <= 4800>

In [6]:
# Sample output

<gurobi.TempConstr: <gurobi.LinExpr: 1300.0 x[1] + 950.0 x[2] + 1000.0 x[3] + 1400.0 x[4] + 1600.0 x[5]> <= 4800>

In [8]:
# Entire formulation (Write your code here)
from gurobipy import Model, GRB
mod=Model()
jobs = range(1,6)
x=mod.addVars(jobs, vtype=GRB.BINARY, name='x')
y=mod.addVars(jobs, vtype=GRB.BINARY, name='y')
mod.update()
mod.setObjective(sum(i*x[j] for i,j in zip(profit.iloc[0],jobs)) + sum(i*y[j] for i,j in zip(profit.iloc[1],jobs)),sense = GRB.MAXIMIZE)
mod.addConstr(sum(b*x[j] for b,j in zip(laborRequired,jobs))<=4800,name='Labor')
for j in jobs:
    mod.addConstr(x[j]+y[j]==1,name=f'Project_{j}')
mod.write('10-books.lp')
!type 10-books.lp

\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  30 x[1] + 10 x[2] + 26 x[3] + 18 x[4] + 20 x[5] + 6 y[1] + 2 y[2] + 8 y[3]
   + 9 y[4] + 4 y[5]
Subject To
 Labor: 1300 x[1] + 950 x[2] + 1000 x[3] + 1400 x[4] + 1600 x[5] <= 4800
 Project_1: x[1] + y[1] = 1
 Project_2: x[2] + y[2] = 1
 Project_3: x[3] + y[3] = 1
 Project_4: x[4] + y[4] = 1
 Project_5: x[5] + y[5] = 1
Bounds
Binaries
 x[1] x[2] x[3] x[4] x[5] y[1] y[2] y[3] y[4] y[5]
End


In [7]:
# Sample output

\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  30 x[1] + 10 x[2] + 26 x[3] + 18 x[4] + 20 x[5] + 6 y[1] + 2 y[2] + 8 y[3]
   + 9 y[4] + 4 y[5]
Subject To
 Labor: 1300 x[1] + 950 x[2] + 1000 x[3] + 1400 x[4] + 1600 x[5] <= 4800
 Project_1: x[1] + y[1] = 1
 Project_2: x[2] + y[2] = 1
 Project_3: x[3] + y[3] = 1
 Project_4: x[4] + y[4] = 1
 Project_5: x[5] + y[5] = 1
Bounds
Binaries
 x[1] x[2] x[3] x[4] x[5] y[1] y[2] y[3] y[4] y[5]
End


### Final code

Use the final version of your formulation to produce the following output.

In [9]:
# Write your code here
from gurobipy import Model, GRB
mod=Model()
jobs = range(1,6)
x=mod.addVars(jobs, vtype=GRB.BINARY, name='x')
y=mod.addVars(jobs, vtype=GRB.BINARY, name='y')
mod.setObjective(sum(i*x[j] for i,j in zip(profit.iloc[0],jobs)) + sum(i*y[j] for i,j in zip(profit.iloc[1],jobs)),sense = GRB.MAXIMIZE)
mod.addConstr(sum(b*x[j] for b,j in zip(laborRequired,jobs))<=4800,name='Labor')
for j in jobs:
    mod.addConstr(x[j]+y[j]==1,name=f'Project_{j}')
mod.setParam('OutputFlag',False)
mod.optimize()
print('Optimal objective:',mod.objVal)
print('Optimal solution: do projects ',end='')
for j in jobs:
    if x[j].x==1:
        print(j, end=' ')
print('yourself')

Optimal objective: 88.0
Optimal solution: do projects 1 2 3 4 yourself


In [8]:
# Sample output

Optimal objective: 88.0
Optimal solution: do projects 1 2 3 4 yourself
